<a href="https://colab.research.google.com/github/ABMHub/NLP/blob/main/T2/preprocess2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicialização

In [107]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import nltk
from nltk.stem.porter import *
from typing import List
import matplotlib.pyplot as plt

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Aula Feature Engineering

In [108]:
corpus = [
          "Batatinha quando nasce esparrama pelo chão",
          "A pior experiência da minha vida",
          "Quero meu dinheiro de volta",
          "A experiência do dinheiro esparrama minha vida",
          "Açaí é a melhor coisa da vida"
]

corpus

['Batatinha quando nasce esparrama pelo chão',
 'A pior experiência da minha vida',
 'Quero meu dinheiro de volta',
 'A experiência do dinheiro esparrama minha vida',
 'Açaí é a melhor coisa da vida']

In [109]:
frasezona = " ".join(corpus).lower()
frasezona

'batatinha quando nasce esparrama pelo chão a pior experiência da minha vida quero meu dinheiro de volta a experiência do dinheiro esparrama minha vida açaí é a melhor coisa da vida'

In [110]:
tokens = frasezona.split()
tokens

['batatinha',
 'quando',
 'nasce',
 'esparrama',
 'pelo',
 'chão',
 'a',
 'pior',
 'experiência',
 'da',
 'minha',
 'vida',
 'quero',
 'meu',
 'dinheiro',
 'de',
 'volta',
 'a',
 'experiência',
 'do',
 'dinheiro',
 'esparrama',
 'minha',
 'vida',
 'açaí',
 'é',
 'a',
 'melhor',
 'coisa',
 'da',
 'vida']

In [111]:
vocab = pd.Series(list(set(tokens)))
vocab.sort_values()

9               a
4            açaí
15      batatinha
17           chão
19          coisa
3              da
18             de
8        dinheiro
12             do
10      esparrama
11    experiência
7          melhor
21            meu
1           minha
5           nasce
20           pelo
6            pior
14         quando
16          quero
2            vida
0           volta
13              é
dtype: object

In [112]:
bow = dict()
for i in range(len(corpus)):
  bow[i] = corpus[i].lower().split()

bow

{0: ['batatinha', 'quando', 'nasce', 'esparrama', 'pelo', 'chão'],
 1: ['a', 'pior', 'experiência', 'da', 'minha', 'vida'],
 2: ['quero', 'meu', 'dinheiro', 'de', 'volta'],
 3: ['a', 'experiência', 'do', 'dinheiro', 'esparrama', 'minha', 'vida'],
 4: ['açaí', 'é', 'a', 'melhor', 'coisa', 'da', 'vida']}

Bag of Words - Binário
0 caso não esteja no texto, 1 caso esteja

In [113]:
bow_binario = dict()

for i, sent in enumerate(corpus):
  bow_binario[i] = dict()
  for wrd in sent.lower().split():
    bow_binario[i][wrd] = 1

df_bin = pd.DataFrame().from_records(bow_binario).T.fillna(0)
df_bin = df_bin.astype(int)
df_bin
# plt.figure(figsize=(21, 4))
# sns.heatmap(df_bin)


,batatinha,quando,nasce,esparrama,pelo,chão,a,pior,experiência,da,...,quero,meu,dinheiro,de,volta,do,açaí,é,melhor,coisa
0,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,0,0,0
3,0,0,0,1,0,0,1,0,1,0,...,0,0,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,1,1,1,1


Bag of words - contagem
Conta quantas vezes cada palavra apareceu no texto

In [114]:
bow_contagem = dict()

for i, sent in enumerate(corpus):
  bow_contagem[i] = dict()
  for wrd in sent.lower().split():
    if wrd in bow_contagem[i].keys():
      bow_contagem[i][wrd] += 1
    else:
      bow_contagem[i][wrd] = 1

df_contagem = pd.DataFrame().from_records(bow_contagem).T.fillna(0)
df_contagem = df_contagem.astype(int)
df_contagem

,batatinha,quando,nasce,esparrama,pelo,chão,a,pior,experiência,da,...,quero,meu,dinheiro,de,volta,do,açaí,é,melhor,coisa
0,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,0,0,0
3,0,0,0,1,0,0,1,0,1,0,...,0,0,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,1,1,1,1


Similaridade de textos: comparação de cosseno

In [115]:
def norma(vec):
  norma = 0
  for x in vec:
    norma += x**2
  norma ** 0.5
  return norma

def similaridade_de_coseno(vec1, vec2):
  prod_interno = 0
  for i in range(len(vec1)):
    prod_interno += vec1[i] * vec2[i]

  norma1 = norma(vec1)
  norma2 = norma(vec2)

  print(prod_interno, norma1, norma2)

  return prod_interno/(norma1*norma2)

In [118]:
similaridade_de_coseno([0,0,1,1], [0, 0, 1, 1])

2 2 2


0.5